In [ ]:
# %tensorboard --logdir ./logs/fit

In [38]:
from datetime import datetime, timedelta
import akshare as ak
import pandas as pd

In [39]:
import sys
sys.path.append('../')

import sqlite3
import database.database_config as config
from database.downloader import DownloaderBase

db_conn = sqlite3.connect('../database/hh_quant.db')
db_downloader = DownloaderBase(db_conn=db_conn, db_config=config)

In [44]:
trade_date = db_downloader._download_stock_trade_date()

In [45]:
trade_date.max()

datetime    2023-12-29
dtype: object

In [43]:
pd.to_datetime(datetime.strptime('20240101', '%Y%m%d'))

Timestamp('2024-01-01 00:00:00')

In [ ]:
datetime.strptime(db_downloader._download_stock_trade_date().max(), "%Y-%m-%d")